In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

sc = SparkSession.builder.getOrCreate()

24/04/10 15:32:26 WARN Utils: Your hostname, Ruths-MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 10.192.238.85 instead (on interface en0)
24/04/10 15:32:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/10 15:32:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
paths = ['data_lake/compravenda_sup.parquet', 'data_lake/rent_price.parquet', 'data_lake/renda.parquet']
col_numeriques = {'data_lake/compravenda_sup.parquet': ['Nombre']
                  , 'data_lake/renda.parquet': ['Import_Euros']
                , 'data_lake/rent_price.parquet': ['Price']}

In [4]:
for path in paths:
    RDD = sc.read.parquet(path)
    RDD.printSchema()

root
 |-- Any: long (nullable = true)
 |-- Trimestre: long (nullable = true)
 |-- Codi_Districte: long (nullable = true)
 |-- Nom_Districte: string (nullable = true)
 |-- Codi_Barri: long (nullable = true)
 |-- Nom_Barri: string (nullable = true)
 |-- Superfície_mitjana_(m2_construïts): string (nullable = true)
 |-- Nombre: string (nullable = true)

root
 |-- Year: long (nullable = true)
 |-- Trimester: long (nullable = true)
 |-- District: string (nullable = true)
 |-- Neighbourhood: string (nullable = true)
 |-- Average _rent: string (nullable = true)
 |-- Price: double (nullable = true)

root
 |-- Any: long (nullable = true)
 |-- Codi_Districte: long (nullable = true)
 |-- Nom_Districte: string (nullable = true)
 |-- Codi_Barri: long (nullable = true)
 |-- Nom_Barri: string (nullable = true)
 |-- Seccio_Censal: long (nullable = true)
 |-- Import_Euros: long (nullable = true)



In [ ]:
# Definir la función que convierte las variables a m2 si están en una magnitud diferente
def convertir_a_m2(magnitud, valor):
    if magnitud.lower() == "m2":
        return valor
    elif magnitud.lower() == "m":
        return valor * valor
    elif magnitud.lower() == "cm2":
        return valor / 10000
    elif magnitud.lower() == "mm2":
        return valor / 1000000
    elif magnitud.lower() == "km2":
        return valor * 1000000
    else:
        return valor

# Registrar la función definida como una función Spark UDF (User Defined Function)
convertir_a_m2_udf = udf(convertir_a_m2, StringType())

In [ ]:
for path in paths:
    RDD = sc.read.parquet(path)
    RDD.printSchema()
    df_converted = RDD.withColumn("valor_en_m2", convertir_a_m2_udf("magnitud", "valor"))